# StyleGAN2-ada operations

In [ ]:
# modules for notebook logging
!pip install IPython
!pip install ipywidgets

## Training

First, let's prepare the data. Ensure all your images have the same color channels (monochrome, RGB or RGBA).  

If you work with patterns or shapes (rather than compostions), you can crop square fragments from bigger images (effectively multiplying their amount). For that, edit source and target paths below; `size` is fragment resolution, `step` is shift between the fragments. This will cut every source image into 512x512px fragments, overlapped with 256px shift by X and Y. Edit `size` and `overlap`  according to your dataset. 

In [ ]:
src_dir = 'data/src'
data_dir = 'data/mydata'
size = 512
step = 256

%run src/util/multicrop.py --in_dir $src_dir --out_dir $data_dir --size $size --step $step

If you edit the images yourself (e.g. for non-square aspect ratios), ensure their correct size and put to the `data_dir`.  
For conditional model split the data by subfolders (`mydata/1`, `mydata/2`, ..) and add `--cond` option to the training command below.

In [ ]:
data_dir = 'data/mydata'

Now, we can train StyleGAN2 on the prepared dataset (which is the image folder itself, no need to convert it to some special format).

In [ ]:
%run src/train.py --data $data_dir --kimg 1000

> This will run training process, according to the options and settings in `src/train.py` (check and explore those!!). Results (models and samples) are saved under `train` directory, similar to original Nvidia approach. There are two types of models saved: compact (containing only Gs network for inference) as `<dataset>-...pkl` (e.g. `mydata-512-0360.pkl`), and full (containing G/D/Gs networks for further training) as `snapshot-...pkl`. 

> The length of the training is defined by `--kimg X` argument (training duration in thousands of images). Reasonable `kimg` value for full training from scratch is 5000-8000, while for finetuning in `--d_aug` mode 1000-2000 may be sufficient.

> NB: If you get `RuntimeError: context has already been set` (probably after training interruption), restart IPython kernel and try again.  
If you have troubles with custom cuda ops, try removing their cached version (`C:\Users\eps\AppData\Local\torch_extensions` on Windows).


If the training process was interrupted, we can resume it from the last saved model as following:  
*(replace `000-mydata-512-...` with existing training directory)*

In [ ]:
%run src/train.py --data $data_dir --resume train/001-mydata-512-auto-gf_bnc --kimg 1000

NB: In most cases it's much easier to use a "transfer learning" trick, rather than perform full training from the scratch. For that, we use existing well-trained model as a starter, and "finetune" (uptrain) it with our data. This works pretty well, even if our dataset is very different from the original model. 

So here is a faster way to train our GAN (presuming we have `ffhq-512.pkl` model already):

In [ ]:
%run src/train.py --data $data_dir --resume train/ffhq-512.pkl --kimg 1000

## Generation

Let's produce some imagery from the original cat model (download it from [here](https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-cat-config-f.pkl) and put to `models` directory).  
More cool models can be found [here](https://github.com/justinpinkney/awesome-pretrained-stylegan2).

In [ ]:
from IPython.display import Image, Video

model = 'models/stylegan2-cat-config-f' # without ".pkl" extension
model_pkl = model + '.pkl' # with ".pkl" extension
output = '_out/cats'
frames = '50-10'

Generate some animation to test the model:

In [ ]:
%run src/_genSGAN2.py --model $model_pkl --out_dir $output --frames $frames

Image(filename = output + '/000000.jpg') # show first frame

> Here we loaded the model 'as is', and produced 50 frames in its native resolution, interpolating between random latent space keypoints, with a step of 10 frames between keypoints.

If you have `ffmpeg` installed, you can convert it into video:

In [ ]:
out_sequence = output + '/%06d.jpg'
out_video = output + '.mp4'

!ffmpeg -y -v warning -i $out_sequence $out_video

Video(out_video)

>
Now let's generate custom animation. For that we omit model extension, so the script would load custom network, effectively enabling special features, e.g. arbitrary resolution (set by `--size` argument in `X-Y` format).  
`--cubic` option changes linear interpolation to cubic for smoother animation (there is also `--gauss` option for additional smoothing).

In [ ]:
%run src/_genSGAN2.py --model $model --out_dir $output --frames $frames --size 400-300 --cubic
Image(output+'/000000.jpg')

>  **Run `ffmpeg` command above after each generation, if you want to check results in motion.**  
Adding `--save_lat` option will save all traversed dlatent points as Numpy array in `*.npy` file (useful for further curating). Set `--seed X` value to produce repeatable results (0 = random).

Generate more various imagery:

In [ ]:
%run src/_genSGAN2.py --model $model --out_dir $output --frames $frames --size 768-256 -n 3-1
Image(output+'/000000.jpg')

> Here we get animated composition of 3 independent frames, blended together horizontally (similar to the image in the repo header). Argument `--splitfine X` controls boundary fineness (0 = smoothest/default, higher => thinner).  

Instead of frame splitting, we can load external mask from b/w image file (it also could be folder with file sequence):

In [ ]:
%run src/_genSGAN2.py --model $model --out_dir $output --frames $frames --size 400-300 --latmask _in/mask.jpg
Image(output+'/000000.jpg')

> 
`--digress X` adds some funky displacements with X strength (by tweaking initial constant layer).  
`--trunc X` controls truncation psi parameter (0 = boring, 1+ = weird). 

In [ ]:
%run src/_genSGAN2.py --model $model --out_dir $output --frames $frames --digress 2 --trunc 1.2 --save_lat
Image(output+'/000000.jpg')

> Don't forget to check other options of `_genSGAN2.py` by `--help` argument.

### Latent space exploration

For these experiments download [FFHQ model](https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-ffhq-config-f.pkl) and save to `models`.

In [ ]:
from IPython.display import Image, Video

model = 'models/stylegan2-ffhq-config-f' # without ".pkl" extension
model_pkl = model + '.pkl' # with ".pkl" extension

> 
Project external images (aligned face portraits) from `_in/photo` onto StyleGAN2 model dlatent space. 
Results (found dlatent points as Numpy arrays in `*.npz` files, and video/still previews) are saved to `_out/proj` directory.

In [ ]:
%run src/projector.py --model $model_pkl --in_dir _in/photo --out_dir _out/proj --save_video 

> 
Generate animation between saved dlatent points:

In [ ]:
dlat = 'dlats'
path_in = '_in/' + dlat
path_out = '_out/ffhq-' + dlat
out_sequence = path_out + '/%06d.jpg'
out_video = path_out + '.mp4'

%run src/_play_dlatents.py --model $model --dlatents $path_in --out_dir $path_out --fstep 10
Image(path_out+'/000000.jpg', width=512, height=512)

!ffmpeg -y -v warning -i $out_sequence $out_video
Video(out_video, width=512, height=512)

> This loads saved dlatent points from `_in/dlats` and produces smooth looped animation between them (with interpolation step of 10 frames). `dlats` may be a file or a directory with `*.npy` or `*.npz` files. To select only few frames from a sequence `somename.npy`, create text file with comma-delimited frame numbers and save it as `somename.txt` in the same directory (check given examples for FFHQ model).

Style-blending argument `--style_dlat blonde458.npy` would also load dlatent from `blonde458.npy` and apply it to higher network layers. `--cubic` smoothing and `--digress X` displacements are also applicable here:

In [ ]:
%run src/_play_dlatents.py --model $model --dlatents $path_in --out_dir $path_out --fstep 10 --style_dlat _in/blonde458.npy --digress 2 --cubic
!ffmpeg -y -v warning -i $out_sequence $out_video
Video(out_video, width=512, height=512)

> 
Generate animation by moving saved dlatent point `_in/blonde458.npy` along feature direction vectors from `_in/vectors_ffhq` (aging/smiling/etc) one by one:

In [ ]:
%run src/_play_vectors.py --model $model_pkl --base_lat _in/blonde458.npy --vector_dir _in/vectors_ffhq --out_dir _out/ffhq_looks

!ffmpeg -y -v warning -i _out/ffhq_looks/%06d.jpg _out/ffhq-vectors.mp4
Video('_out/ffhq-vectors.mp4', width=512, height=512)

> Such vectors can be discovered, for example, with the following methods:
> * https://github.com/genforce/sefa
> * https://github.com/harskish/ganspace
> * https://github.com/LoreGoetschalckx/GANalyze